## Inverse Kinematics Calculation Notebook

First, import all library modules.

In [6]:
# import modules
import tf
import transformations
import numpy as np
#from trajectory_msgs.msg import JointTrajectory, JointTrajectoryPoint
#from geometry_msgs.msg import Pose
from mpmath import *
from sympy import *
import pickle
import math


### DH Parameters

Define DH Parameters for Kuka arm

In [7]:
q1, q2, q3, q4, q5, q6, q7 = symbols('q1:8')
d1, d2, d3, d4, d5, d6, d7 = symbols('d1:8')
a0, a1, a2, a3, a4, a5, a6 = symbols('a0:7')
alpha0, alpha1, alpha2, alpha3, alpha4, alpha5, alpha6 = symbols('alpha0:7')

s = {alpha0:        0,  a0:      0, d1: 0.75, 
                 alpha1: -np.pi/2,  a1:   0.35, d2:    0, q2: q2-np.pi/2,
                 alpha2:        0,  a2:   1.25, d3:    0, 
                 alpha3: -np.pi/2,  a3: -0.054, d4:  1.5, 
                 alpha4:  np.pi/2,  a4:      0, d5:    0, 
                 alpha5: -np.pi/2,  a5:      0, d6:    0, 
                 alpha6:        0,  a6:      0, d7:0.303, q7:          0}


## Generate Forward Kinematics Function

Generate the Forward Kinematics Functions and save them in pickle files.
Uncomment the last line to run the generator.


In [8]:
def generate_forward():

            # Modified DH params


            
            # Define Modified DH Transformation matrix
            T0_1 = Matrix([[             cos(q1),            -sin(q1),            0,              a0],
                           [ sin(q1)*cos(alpha0), cos(q1)*cos(alpha0), -sin(alpha0), -sin(alpha0)*d1],
                           [ sin(q1)*sin(alpha0), cos(q1)*sin(alpha0),  cos(alpha0),  cos(alpha0)*d1],
                           [                   0,                   0,            0,               1]])
            T0_1 = T0_1.subs(s)

            T1_2 = Matrix([[             cos(q2),            -sin(q2),            0,              a1],
                           [ sin(q2)*cos(alpha1), cos(q2)*cos(alpha1), -sin(alpha1), -sin(alpha1)*d2],
                           [ sin(q2)*sin(alpha1), cos(q2)*sin(alpha1),  cos(alpha1),  cos(alpha1)*d2],
                           [                   0,                   0,            0,               1]])
            T1_2 = T1_2.subs(s)

            T2_3 = Matrix([[             cos(q3),            -sin(q3),            0,              a2],
                           [ sin(q3)*cos(alpha2), cos(q3)*cos(alpha2), -sin(alpha2), -sin(alpha2)*d3],
                           [ sin(q3)*sin(alpha2), cos(q3)*sin(alpha2),  cos(alpha2),  cos(alpha2)*d3],
                           [                   0,                   0,            0,               1]])
            T2_3 = T2_3.subs(s)

            T3_4 = Matrix([[             cos(q4),            -sin(q4),            0,              a3],
                           [ sin(q4)*cos(alpha3), cos(q4)*cos(alpha3), -sin(alpha3), -sin(alpha3)*d4],
                           [ sin(q4)*sin(alpha3), cos(q4)*sin(alpha3),  cos(alpha3),  cos(alpha3)*d4],
                           [                   0,                   0,            0,               1]])
            T3_4 = T3_4.subs(s)

            T4_5 = Matrix([[             cos(q5),            -sin(q5),            0,              a4],
                           [ sin(q5)*cos(alpha4), cos(q5)*cos(alpha4), -sin(alpha4), -sin(alpha4)*d5],
                           [ sin(q5)*sin(alpha4), cos(q5)*sin(alpha4),  cos(alpha4),  cos(alpha4)*d5],
                           [                   0,                   0,            0,               1]])
            T4_5 = T4_5.subs(s)

            T5_6 = Matrix([[             cos(q6),            -sin(q6),            0,              a5],
                           [ sin(q6)*cos(alpha5), cos(q6)*cos(alpha5), -sin(alpha5), -sin(alpha5)*d6],
                           [ sin(q6)*sin(alpha5), cos(q6)*sin(alpha5),  cos(alpha5),  cos(alpha5)*d6],
                           [                   0,                   0,            0,               1]])
            T5_6 = T5_6.subs(s)

            T6_7 = Matrix([[             cos(q7),            -sin(q7),            0,              a6],
                           [ sin(q7)*cos(alpha6), cos(q7)*cos(alpha6), -sin(alpha6), -sin(alpha6)*d7],
                           [ sin(q7)*sin(alpha6), cos(q7)*sin(alpha6),  cos(alpha6),  cos(alpha6)*d7],
                           [                   0,                   0,            0,               1]])
            T6_7 = T6_7.subs(s)

            # Create individual transformation matrices
            T0_2 = simplify(T0_1 * T1_2)
            print("done T0_2")
            T0_3 = simplify(T0_2 * T2_3)
            print("done T0_3")
            T0_4 = simplify(T0_3 * T3_4)
            print("done T0_4")
            T0_5 = simplify(T0_4 * T4_5)
            print("done T0_5")
            T0_6 = simplify(T0_5 * T5_6)
            print("done T0_6")
            T0_7 = simplify(T0_6 * T6_7)
            print("done T0_7")
            
            # correction for URDF and DH Convention for Gripper
            # 180 degrees on the Z axis
            R_z = Matrix([[           cos(np.pi),         -sin(np.pi),            0,              0],
                           [          sin(np.pi),          cos(np.pi),            0,              0],
                           [                   0,                   0,            1,              0],
                           [                   0,                   0,            0,              1]])
            # 90 degrees on the Y axis
            R_y = Matrix([[        cos(-np.pi/2),                   0,sin(-np.pi/2),              0],
                           [                   0,                   1,            0,              0],
                           [      -sin(-np.pi/2),                   0,cos(-np.pi/2),              0],
                           [                   0,                   0,            0,              1]])
            R_correction = simplify(R_z * R_y)

            T_final = simplify(T0_7 * R_correction)

            R0_3 = simplify(T0_1*T1_2*T2_3)[:3,:3]
            R3_6 = simplify(T3_4*T4_5*T5_6)[:3,:3]
            
            pickle.dump( T0_1, open( "T0_1.pw", "wb" ) )
            pickle.dump( T0_2, open( "T0_2.pw", "wb" ) )
            pickle.dump( T0_3, open( "T0_3.pw", "wb" ) )
            pickle.dump( T0_4, open( "T0_4.pw", "wb" ) )
            pickle.dump( T0_5, open( "T0_5.pw", "wb" ) )
            pickle.dump( T0_6, open( "T0_6.pw", "wb" ) )
            pickle.dump( T0_7, open( "T0_7.pw", "wb" ) )
            pickle.dump( T_final, open( "T_final.pw", "wb" ) )
            pickle.dump( R0_3, open( "R0_3.pw", "wb" ) )
            pickle.dump( R3_6, open( "R3_6.pw", "wb" ) )
            
            return None

#generate_forward()

## Load Forward Kinematics pickle files

Run the previous function to generate the files if needed.

In [9]:

T0_1 = pickle.load( open( "T0_1.pw", "rb" ) )
T0_2 = pickle.load( open( "T0_2.pw", "rb" ) )
T0_3 = pickle.load( open( "T0_3.pw", "rb" ) )
T0_4 = pickle.load( open( "T0_4.pw", "rb" ) )
T0_5 = pickle.load( open( "T0_5.pw", "rb" ) )
T0_6 = pickle.load( open( "T0_6.pw", "rb" ) )
T0_7 = pickle.load( open( "T0_7.pw", "rb" ) )
T_final = pickle.load( open( "T_final.pw", "rb" ) )
R0_3 = pickle.load( open( "R0_3.pw", "rb" ) )
R3_6 = pickle.load( open( "R3_6.pw", "rb" ) )


### Find Wrist Center WC = O5

#### Rest Position

At the rest position pose = (2.153, 0, 1.947, 0, 0, 0, 1)

WC position is (1.850, 0, 1.947, 0, 0, 0, 1)

D6 + End-effector Length l = 0.303

#### Joint 5 theta5 = pi /2

pose = (1.851, 0, 1.643, 0, 0.706, 0, 0.708) RPY (0, 1.567, 0)

WC Position is (1.850, 0, 1.947, 0, 0706, 0, 0.708)

#### Joint 5 theta5 = -pi /2

pose = (1.851, 0, 2.249, 0, -0.706, 0.025, 0.708) RPY (0.07, -1.568, 0)

WC Position is (1.850, 0, 1.947, 0, 0706, 0, 0.708)

#### Joint 5 theta5 = 0.32

pose = (2.137, 0, 1.850, 0, 0.161, 0.0, 0.987) RPY (0.0, 0.323, 0)

WC Position is (1.850, 0, 1.947, 0, 0.161, 0, 0.987)

#### thata1 = 0.8, thata2 = 0.4, 

pose = (1.728, 1.772, 1.157, -0.076, 0.181, 0.381, 0.903) RPY (0.0, 0.396, 0.798)

WC Position is (1.533, 1.572, 1.274, -0.076, 0.181, 0.381, 0.903)



#### thata1 = -0.4, thata2 = -0.4, 

pose = (1.396, -0.598, 2.571, -0.041, -0.202, -0.197, 0.959) RPY (0.0, 0.415, -0.405)

WC Position is (1.141, -0.489, 2.449, -0.041, -0.202, -0.197, 0.959)



#### thata3 = -0.91

pose = (1.499, 0.0, 3.39, 0.0, -0.439, 0, 0.898)

WC Position is (1.313, 0.0, 3.151, 0.0, -0.439, 0, 0.898)



In [10]:
# (pose[4], pose[5], pose[6]) is the unit vector for the direction

def cal_wc(pose):
    px = pose[0]
    py = pose[1]
    pz = pose[2]
    (roll, pitch, yaw) = transformations.euler_from_quaternion([pose[3],pose[4], pose[5], pose[6]], 'sxyz')
    #print(roll, pitch, yaw)
    L = 0.303
    
    Rrpy_urdf = transformations.euler_matrix(roll,pitch,yaw,'sxyz')
    
    R_z = Matrix([[           cos(np.pi),         -sin(np.pi),            0,              0],
                           [          sin(np.pi),          cos(np.pi),            0,              0],
                           [                   0,                   0,            1,              0],
                           [                   0,                   0,            0,              1]])
    # 90 degrees on the Y axis
    R_y = Matrix([[        cos(-np.pi/2),                   0,sin(-np.pi/2),              0],
                           [                   0,                   1,            0,              0],
                           [      -sin(-np.pi/2),                   0,cos(-np.pi/2),              0],
                           [                   0,                   0,            0,              1]])
    R_correction = R_z * R_y

    Rrpy_4 = N(Rrpy_urdf * R_correction)
    #print(t06r)

    Rrpy = Rrpy_4[:3,:3]
    #print(t06r_3)
    
    EE = Matrix([[px],[py],[pz]])
    
    #print(r06corr_3*Matrix([[1],[0],[0]]))
    
    WC = EE - L * (Rrpy*Matrix([[0],[0],[1]]))
    
    #print("WCx = ", WC[0])
    
    return WC, Rrpy
    
#pose = (2.153, 0, 1.947, 0, 0, 0, 1)
#pose =  (1.851, 0, 1.643, 0, 0.706, 0, 0.708)
#pose = (1.851, 0, 2.249, 0, -0.706, 0.025, 0.708)
#pose = (2.137, 0, 1.850, 0, 0.161, 0.0, 0.987)
#pose = (0.904, 1.744, 0.487, -0.281, 0.462, 0.437, 0.719)
#pose = (1.728, 1.772, 1.157, -0.076, 0.181, 0.381, 0.903)
pose = (1.425, -0.611, 2.551, -0.040, -0.193, -0.197, 0.960)
pose = (1.499, 0.0, 3.39, 0.0, -0.439, 0, 0.898)

cal_wc(pose)

(Matrix([
 [    1.31289120580508],
 [3.71067980141648e-17],
 [    3.15089224871763]]), Matrix([
 [  -0.789134492681096, -7.52203099353079e-17,     0.614220442887527],
 [7.49879891330929e-33,                  -1.0, -1.22464679914735e-16],
 [   0.614220442887527, -9.66411030558674e-17,     0.789134492681096]]))

## Find the angles of the first 3 joints base on WC location



In [11]:
# calculate to WC = Orig5
def ik_cal_to_wc(WC):
            px = WC[0]
            py = WC[1]
            pz = WC[2]
            
            # Find theta1 base on end-effector position project to ground plane (pz = 0)
            # joint1 limit: -185 degree to 185 degree
            theta1 = math.atan2(py, px)

            # joint2 limit: -45 degree to 85 degree
            #               -0.78 to 1.48
            j2_lower_rad = np.deg2rad(-45.0)
            j2_upper_rad = np.deg2rad(85.0)

            # joint3 limit: -210 degree to 155-90 = 65 degree
            #               -3.66 to 1.13
            j3_lower_rad = np.deg2rad(-210.0)
            j3_upper_rad = np.deg2rad(65.0)

            # distance from Orig2 to P
            orig2z = 0.75  # d1
            orig2x = 0.35 * cos(theta1)  # a1
            orig2y = 0.35 * sin(theta1)

            O2P = sqrt( np.square(px - orig2x) + np.square(py - orig2y) + np.square(pz - orig2z))

            O2Pz = orig2z - pz

            # distance from Orig3 to Orig5 is fixed
            # a1 = 0.35, d4 = 1.5 => 1.85
            O3P = sqrt( np.square(1.85 - 0.35) + np.square(1.946 - 2.0) )

            # angle g1 is the angle drop below X-Y plane from O2
            g1 = asin(O2Pz / O2P)
            #print("g1 = ",g1)

            # a2 = 1.25
            f1 = acos( (np.square(1.25) + np.square(O2P) - np.square(O3P)) / (2 * 1.25 * O2P))

            theta2 = np.pi/2 + g1 - f1

            # a2 = 1.25
            orig3z = orig2z + (1.25 * cos(theta2))

            # need to offset by a3 = -0.054
            s = orig3z - 0.054 - pz

            #theta3 = atan2(s, O3P)
            theta3 = asin(s / O3P) - theta2

            #print("theta1 = ",theta1)
            #print("theta2 = ",theta2)
            #print("theta3 = ",theta3)

            return theta1, theta2, theta3

#ik_cal_to_wc(1.850, 0, 1.947)
ik_cal_to_wc([1.313, 0.0, 3.151])
ik_cal_to_wc([1.168, -0.500, 2.433])

#print(R3_6)

(-0.40447839234418426, -0.397272915568273, -0.00261472718263545)

## SImplified Symbol Equation of R3_6


R3_6 = 

([[-sin(q4)*sin(q6) + cos(q4)*cos(q5)*cos(q6), -sin(q4)*cos(q6) - sin(q6)*cos(q4)*cos(q5), -sin(q5)*cos(q4)],

  [                           sin(q5)*cos(q6),                           -sin(q5)*sin(q6),          cos(q5)],
  
  [-sin(q4)*cos(q5)*cos(q6) - sin(q6)*cos(q4),  sin(q4)*sin(q6)*cos(q5) - cos(q4)*cos(q6),  sin(q4)*sin(q5)]])
  

## Function to calculate R0_3 and Final Pose using Forward Kinematics


In [12]:
def forward_kinematics_R03_inv(cq1, cq2, cq3, cq4, cq5, cq6):
            Till3 = T0_3.evalf(subs={q1: cq1, q2: cq2, q3: cq3, q4: cq4, q5: cq5, q6: cq6})
            R0_3 = Till3[0:3,0:3]
            Inv_R0_3 = R0_3.pinv()

            return Inv_R0_3

# if run transformations.quaternion_from_matrix(final) directly, we'll get
# TypeError: __array__() takes 1 positional argument but 2 were given
def quad_from_matrix(M):
            f = np.zeros((4,4))
            f[0,0] = M[0,0]
            f[0,1] = M[0,1]
            f[0,2] = M[0,2]
            f[1,0] = M[1,0]
            f[1,1] = M[1,1]
            f[1,2] = M[1,2]
            f[2,0] = M[2,0]
            f[2,1] = M[2,1]
            f[2,2] = M[2,2]
            f[3,3] = 1

            quad_final = transformations.quaternion_from_matrix(f)
            return quad_final

def forward_kinematics(Thetas):
            cq1 = Thetas[0]
            cq2 = Thetas[1]
            cq3 = Thetas[2]
            cq4 = Thetas[3]
            cq5 = Thetas[4]
            cq6 = Thetas[5]
            final = T_final.evalf(subs={q1: cq1, q2: cq2, q3: cq3, q4: cq4, q5: cq5, q6: cq6})

            #print("Final link")
            quad = quad_from_matrix(final)

            fpose = (final[0,3], final[1,3], final[2,3], quad[0], quad[1], quad[2], quad[3])
            return fpose


## Inverse Kinematics Function to calculate all joint angles

This is the main function that given an input pose, returns a list of all 6 angles for the joints.

In [13]:
# thata1 = -0.4, thata2 = -0.4,
# RPY (0.0, 0.415, -0.405)

def cal_all_angles(pose):
    WC, Rrpy = cal_wc(pose)

    theta1, theta2, theta3 = ik_cal_to_wc(WC)

    R03_inv = forward_kinematics_R03_inv(theta1, theta2, theta3, 0, 0, 0)
    R36 = R03_inv * Rrpy

    # cos(theta5) = R36[2,1]
    # sin(theta5) * cos(theta6) = R36[1,0]
    # -sin(theta5) * cos(theta4) = R36[0,2]
    #theta5 = acos(round(R36[1,2],2))
    theta5 = acos(R36[1,2])
    if abs(theta5) < 0.01:
        theta6 = 0
        theta4 = acos(-R36[2,1])
    else:
        theta6 = acos(R36[1,0] / sin(theta5))
        theta4 = acos(-(R36[0,2] / sin(theta5)))

    Thetas = (theta1, theta2, theta3, theta4, theta5, theta6)
    return Thetas



## Test using a few sample pose

Test the function with a sample pose, and see the result and error.

In [14]:
pose = (1.396, -0.598, 2.571, -0.041, -0.202, -0.197, 0.959) 
pose = (0.555, 0.679, 2.486, 0.428, 0.260, 0.864, 0.041) 
Thetas = cal_all_angles(pose)
fpose = forward_kinematics(Thetas)
print(fpose)
print(Thetas)
mse = ((np.array(fpose) - np.array(pose)) ** 2).mean()
print("Error:",mse)

(0.553682523378719, 0.677957817040109, 2.49021997061804, 0.42847163341177869, 0.41449009383127317, 0.80196052746224877, -0.038331891026354509)
(0.6692565182937807, -0.688659682851807, 0.272319795234066, 2.47138724746555, 1.56517051099893, 0.184727357051625)
Error: 0.00486149809409101
